In [1]:
import pandas as pd

In [5]:
#Global paramenters
#API URLs from RxNorm
base_url = "https://rxnav.nlm.nih.gov/REST/rxcui/"
base_url_2 = "https://rxnav.nlm.nih.gov/REST/"
base_url_3 = "https://rxnav.nlm.nih.gov/REST/rxcui.json?name="
#RxNorm methods
getAllRelatedInfo = "/allrelated.json"
getRelatedByType = "/related.json?tty=IN"
#dataframe
df = pd.DataFrame()

In [6]:
#Method to get drug with only the opioid ingredient
def query_self(name_of_drug):
    rd_df = pd.read_json(base_url+name_of_drug+getRelatedByType)
    rd_df = pd.DataFrame(rd_df.relatedGroup.conceptGroup)
    rd_df = pd.DataFrame(rd_df.conceptProperties[0])
    return rd_df

In [7]:
#Method to get all drugs with an opioid ingredient
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    rd_df = pd.read_json(base_url+name_of_drug+getAllRelatedInfo)
    rd_df = pd.DataFrame(rd_df.allRelatedGroup.conceptGroup)
    rd_df = rd_df.loc[rd_df["tty"] != "DF"].reset_index(drop=True)
    rd_df = rd_df.loc[rd_df["tty"] != "DFG"].reset_index(drop=True)
    while row < rd_df.shape[0]:
        if(pd.isna(rd_df.conceptProperties[row]) is True):
            row+=1
        else:
            new_df = pd.DataFrame(rd_df.conceptProperties[row])
            if df.empty:
                df = new_df
            else:
                df = pd.concat([df, new_df]).reset_index(drop=True)
            row += 1
    return df

In [8]:
antidep_ingredients = pd.read_excel("Antidepressants_Ingredients.xlsx", header=None)

In [9]:
antidep_ingredients

,0
0,5-hydroxytryptophan
1,amineptin
2,amitriptyline
3,amoxapine
4,bifemelane
5,brexanolone
6,bupropion
7,butriptyline
8,citalopram
9,clomipramine


All Antidepressants Ingredients

In [10]:
drugs = []
for drug in antidep_ingredients[0]:
    ingredient = pd.read_json(base_url_3+drug)
    drugs.append(ingredient.idGroup.rxnormId[0])


In [11]:
len(drugs)

58

In [12]:
#loop array to get complete list of drugs
df_antidep = pd.DataFrame()
for drug in drugs:
    self_df = query_self(drug)
    ingredient = self_df["name"].values[0]
    self_df["ingredient"] = ingredient
    merg_df = query_drug(drug)
    merg_df["ingredient"] = ingredient
    df_antidep = pd.concat([df_antidep, self_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
    df_antidep = pd.concat([df_antidep, merg_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_antidep

,rxcui,name,synonym,tty,language,suppress,umlscui,ingredient
0,94,5-hydroxytryptophan,oxitriptan,IN,ENG,N,,5-hydroxytryptophan
1,1293415,Somnicin,,BN,ENG,N,,5-hydroxytryptophan
2,1008781,5-hydroxytryptophan / vitamin B6,,MIN,ENG,N,,5-hydroxytryptophan
3,1293412,5-hydroxytryptophan / magnesium oxide / melato...,,MIN,ENG,N,,5-hydroxytryptophan
4,1945211,5-hydroxytryptophan / melatonin,,MIN,ENG,N,,5-hydroxytryptophan
...,...,...,...,...,...,...,...,...
1645,1439811,vortioxetine 5 MG,,SCDC,ENG,N,,vortioxetine
1646,1439839,vortioxetine 15 MG,,SCDC,ENG,N,,vortioxetine
1647,1439807,vortioxetine Oral Tablet,,SCDF,ENG,N,,vortioxetine
1648,1439805,vortioxetine Oral Product,,SCDG,ENG,N,,vortioxetine


In [13]:
#This list was created by pulling from the RxNorm API
df_antidep = df_antidep[['rxcui', 'name',"tty", "ingredient"]]
df_antidep

,rxcui,name,tty,ingredient
0,94,5-hydroxytryptophan,IN,5-hydroxytryptophan
1,1293415,Somnicin,BN,5-hydroxytryptophan
2,1008781,5-hydroxytryptophan / vitamin B6,MIN,5-hydroxytryptophan
3,1293412,5-hydroxytryptophan / magnesium oxide / melato...,MIN,5-hydroxytryptophan
4,1945211,5-hydroxytryptophan / melatonin,MIN,5-hydroxytryptophan
...,...,...,...,...
1645,1439811,vortioxetine 5 MG,SCDC,vortioxetine
1646,1439839,vortioxetine 15 MG,SCDC,vortioxetine
1647,1439807,vortioxetine Oral Tablet,SCDF,vortioxetine
1648,1439805,vortioxetine Oral Product,SCDG,vortioxetine


In [14]:
#Export
df_antidep.to_excel("New_Complete_Antidepressants_List.xlsx", index=False)

In [15]:
df_list = pd.read_excel("Complete_Antidepressants_List.xlsx")
df_list = df_list[['Code', 'Description','TTY']]
df_list.rename(columns= {'Code' : 'rxcui', 'Description': 'name'}, inplace = True)
df_list

,rxcui,name,TTY
0,1000048,doxepin 10 MG Oral Capsule,SCD
1,1000054,doxepin 10 MG/ML Oral Solution,SCD
2,1000058,doxepin 100 MG Oral Capsule,SCD
3,1000064,doxepin 150 MG Oral Capsule,SCD
4,1000070,doxepin 25 MG Oral Capsule,SCD
...,...,...,...
202,846386,lithium orotate 5 MG Oral Capsule,SCD
203,966787,doxepin 3 MG Oral Tablet,SCD
204,966793,doxepin 6 MG Oral Tablet,SCD
205,1439840,vortioxetine 15 MG Oral Tablet,SCD


In [16]:
#Identify new drugs found
same_drug_count = 0
new_drug_count = 0
drug_not_found_count = 0
same_drug = pd.DataFrame({"drugs_in_list_rxcui": [], "drugs_in_list_name": []})
new_drug = pd.DataFrame({"new_drugs_rxcui": [], "new_drugs_name": []})
for code in df_antidep['rxcui']:
    drug_name = df_antidep['name'].where(df_antidep['rxcui'] == code).dropna().values[0]
    if df_list.isin([int(code)]).any().any() == True:
        same_drug_count += 1
        same_drug = same_drug.append({'drugs_in_list_rxcui': code, 'drugs_in_list_name' : drug_name}, ignore_index = True)
    else:
        new_drug_count += 1
        new_drug = new_drug.append({'new_drugs_rxcui': code, 'new_drugs_name': drug_name}, ignore_index = True)
print('Same drug count: ', same_drug_count)
print('New drug count: ', new_drug_count)
#Displays same and new drugs
df_final = same_drug.join(new_drug)
df_final.head()

C:\Users\Daniel\AppData\Local\Temp\ipykernel_37620\434568379.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_drug = new_drug.append({'new_drugs_rxcui': code, 'new_drugs_name': drug_name}, ignore_index = True)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_37620\434568379.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_drug = new_drug.append({'new_drugs_rxcui': code, 'new_drugs_name': drug_name}, ignore_index = True)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_37620\434568379.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_drug = new_drug.append({'new_drugs_rxcui': code, 'new_drugs_name': drug_name}, ignore_index = True)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_37620\434568379.py:14: FutureWarni

Same drug count:  194
New drug count:  1456


C:\Users\Daniel\AppData\Local\Temp\ipykernel_37620\434568379.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_drug = new_drug.append({'new_drugs_rxcui': code, 'new_drugs_name': drug_name}, ignore_index = True)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_37620\434568379.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_drug = new_drug.append({'new_drugs_rxcui': code, 'new_drugs_name': drug_name}, ignore_index = True)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_37620\434568379.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  same_drug = same_drug.append({'drugs_in_list_rxcui': code, 'drugs_in_list_name' : drug_name}, ignore_index = True)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_37620\434568379.py:11: 

,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name
0,856706,amitriptyline hydrochloride 10 MG / perphenazi...,94,5-hydroxytryptophan
1,856720,amitriptyline hydrochloride 10 MG / perphenazi...,1293415,Somnicin
2,856762,amitriptyline hydrochloride 100 MG Oral Tablet,1008781,5-hydroxytryptophan / vitamin B6
3,856769,amitriptyline hydrochloride 12.5 MG / chlordia...,1293412,5-hydroxytryptophan / magnesium oxide / melato...
4,856773,amitriptyline hydrochloride 150 MG Oral Tablet,1945211,5-hydroxytryptophan / melatonin


In [65]:
absent_drugs = pd.DataFrame({"drugs_not_found_rxcui": [], "drugs_not_found_name": [], "drugs_not_found_tty": []})
count = 0
#for every code from the combind excel list
for code in df_list['rxcui']:
    drug_name = df_list['name'].where(df_list['rxcui'] == code).dropna().values[0]
    tty = df_list['TTY'].where(df_list['rxcui'] == code).dropna().values[0]
    if df_final.isin([str(code)]).any().any() == True:
        count+=1
    else:
        absent_drugs = absent_drugs.append({'drugs_not_found_rxcui': str(code), 'drugs_not_found_name': drug_name, 'drugs_not_found_tty': tty}, ignore_index = True)
print('Count: ', count)
absent_drugs.head()

Count:  194


C:\Users\Daniel\AppData\Local\Temp\ipykernel_21572\1966579902.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  absent_drugs = absent_drugs.append({'drugs_not_found_rxcui': str(code), 'drugs_not_found_name': drug_name, 'drugs_not_found_tty': tty}, ignore_index = True)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_21572\1966579902.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  absent_drugs = absent_drugs.append({'drugs_not_found_rxcui': str(code), 'drugs_not_found_name': drug_name, 'drugs_not_found_tty': tty}, ignore_index = True)
C:\Users\Daniel\AppData\Local\Temp\ipykernel_21572\1966579902.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  absent_drugs = absent_drugs.append({'drugs_not_found_rxcui': str(code), 'd

,drugs_not_found_rxcui,drugs_not_found_name,drugs_not_found_tty
0,197889,lithium carbonate 300 MG Oral Capsule,SCD
1,197890,lithium carbonate 300 MG Oral Tablet,SCD
2,197891,lithium carbonate 300 MG Extended Release Oral...,SCD
3,197892,lithium carbonate 450 MG Extended Release Oral...,SCD
4,197893,lithium carbonate 600 MG Oral Capsule,SCD


In [66]:
df_final = pd.DataFrame()
df_final = same_drug.join(new_drug)
df_final = df_final.join(absent_drugs)
print(df_final.shape) #check if correct size
df_final.head()

(194, 7)


,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name,drugs_not_found_rxcui,drugs_not_found_name,drugs_not_found_tty
0,856706,amitriptyline hydrochloride 10 MG / perphenazi...,94,5-hydroxytryptophan,197889,lithium carbonate 300 MG Oral Capsule,SCD
1,856720,amitriptyline hydrochloride 10 MG / perphenazi...,1293415,Somnicin,197890,lithium carbonate 300 MG Oral Tablet,SCD
2,856762,amitriptyline hydrochloride 100 MG Oral Tablet,1008781,5-hydroxytryptophan / vitamin B6,197891,lithium carbonate 300 MG Extended Release Oral...,SCD
3,856769,amitriptyline hydrochloride 12.5 MG / chlordia...,1293412,5-hydroxytryptophan / magnesium oxide / melato...,197892,lithium carbonate 450 MG Extended Release Oral...,SCD
4,856773,amitriptyline hydrochloride 150 MG Oral Tablet,1945211,5-hydroxytryptophan / melatonin,197893,lithium carbonate 600 MG Oral Capsule,SCD


In [67]:
df_final

,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name,drugs_not_found_rxcui,drugs_not_found_name,drugs_not_found_tty
0,856706,amitriptyline hydrochloride 10 MG / perphenazi...,94,5-hydroxytryptophan,197889,lithium carbonate 300 MG Oral Capsule,SCD
1,856720,amitriptyline hydrochloride 10 MG / perphenazi...,1293415,Somnicin,197890,lithium carbonate 300 MG Oral Tablet,SCD
2,856762,amitriptyline hydrochloride 100 MG Oral Tablet,1008781,5-hydroxytryptophan / vitamin B6,197891,lithium carbonate 300 MG Extended Release Oral...,SCD
3,856769,amitriptyline hydrochloride 12.5 MG / chlordia...,1293412,5-hydroxytryptophan / magnesium oxide / melato...,197892,lithium carbonate 450 MG Extended Release Oral...,SCD
4,856773,amitriptyline hydrochloride 150 MG Oral Tablet,1945211,5-hydroxytryptophan / melatonin,197893,lithium carbonate 600 MG Oral Capsule,SCD
...,...,...,...,...,...,...,...
189,1086784,vilazodone hydrochloride 40 MG Oral Tablet,374620,citalopram Oral Solution,NaN,NaN,NaN
190,1439808,vortioxetine 10 MG Oral Tablet,645288,citalopram Disintegrating Oral Tablet,NaN,NaN,NaN
191,1439810,vortioxetine 20 MG Oral Tablet,730439,citalopram Oral Capsule,NaN,NaN,NaN
192,1439812,vortioxetine 5 MG Oral Tablet,1152130,citalopram Oral Liquid Product,NaN,NaN,NaN


In [68]:
#find the status of the not found drugs
df_status = pd.DataFrame()
index = 0
for code in absent_drugs['drugs_not_found_rxcui']:
    getRxcuiHistoryStatus = 'rxcui/{}/historystatus.json'.format(code)
    df_type = pd.read_json(base_url_2+getRxcuiHistoryStatus)
    df_type = pd.DataFrame(df_type.rxcuiStatusHistory.metaData, index=[0])
    df_type['rxcui'] = code
    df_status = pd.concat([df_status, df_type]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)

In [70]:
df_status

,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate,rxcui
0,Active,RXNORM,042005,,YES,052006,,,197889
1,Active,RXNORM,042005,,YES,052006,,,197890
2,Active,RXNORM,042005,,YES,052006,,,197891
3,Active,RXNORM,042005,,YES,052006,,,197892
4,Active,RXNORM,042005,,YES,052006,,,197893
5,Active,RXNORM,042005,,YES,052006,,,199324
6,Active,RXNORM,042005,,YES,052006,,,311355
7,Active,RXNORM,022006,,YES,052006,,,608328
8,Active,RXNORM,072006,,YES,072006,,,636664
9,Active,RXNORM,012008,,YES,012008,,,746073


In [71]:
absent_drugs = absent_drugs.rename(columns={"drugs_not_found_rxcui": "rxcui"})
absent_drugs

,rxcui,drugs_not_found_name,drugs_not_found_tty
0,197889,lithium carbonate 300 MG Oral Capsule,SCD
1,197890,lithium carbonate 300 MG Oral Tablet,SCD
2,197891,lithium carbonate 300 MG Extended Release Oral...,SCD
3,197892,lithium carbonate 450 MG Extended Release Oral...,SCD
4,197893,lithium carbonate 600 MG Oral Capsule,SCD
5,199324,lithium carbonate 250 MG Oral Tablet,SCD
6,311355,lithium carbonate 150 MG Oral Capsule,SCD
7,608328,rasagiline 1 MG Oral Tablet,SCD
8,636664,rasagiline 0.5 MG Oral Tablet,SCD
9,746073,lithium aspartate 5 MG Oral Capsule,SCD


In [72]:
df_not_found = absent_drugs.merge(df_status, on='rxcui')

In [73]:
df_not_found

,rxcui,drugs_not_found_name,drugs_not_found_tty,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate
0,197889,lithium carbonate 300 MG Oral Capsule,SCD,Active,RXNORM,042005,,YES,052006,,
1,197890,lithium carbonate 300 MG Oral Tablet,SCD,Active,RXNORM,042005,,YES,052006,,
2,197891,lithium carbonate 300 MG Extended Release Oral...,SCD,Active,RXNORM,042005,,YES,052006,,
3,197892,lithium carbonate 450 MG Extended Release Oral...,SCD,Active,RXNORM,042005,,YES,052006,,
4,197893,lithium carbonate 600 MG Oral Capsule,SCD,Active,RXNORM,042005,,YES,052006,,
5,199324,lithium carbonate 250 MG Oral Tablet,SCD,Active,RXNORM,042005,,YES,052006,,
6,311355,lithium carbonate 150 MG Oral Capsule,SCD,Active,RXNORM,042005,,YES,052006,,
7,608328,rasagiline 1 MG Oral Tablet,SCD,Active,RXNORM,022006,,YES,052006,,
8,636664,rasagiline 0.5 MG Oral Tablet,SCD,Active,RXNORM,072006,,YES,072006,,
9,746073,lithium aspartate 5 MG Oral Capsule,SCD,Active,RXNORM,012008,,YES,012008,,
